<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
# Load Data
import pandas as pd 
df = pd.read_csv("./telco.csv") 
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.shape

(7043, 21)

In [3]:
df = df.drop(['customerID'], axis=1)

In [4]:
# Convert Y/N to 0/1

columns = ['Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
           'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
for col in columns:
  df[col] = [1 if x=='Yes' else 0 for x in df[col]]

df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,Male,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [5]:
df['InternetService'] = [0 if x=='No' else 1 for x in df['InternetService']]

In [6]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,1,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,1,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,Male,0,0,0,2,1,No,1,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,1,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,1,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [10]:
df.TotalCharges.replace([" "], ["0"], inplace= True)
df.TotalCharges = df.TotalCharges.astype(float)
data = pd.get_dummies(df)
data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
1,0,0,0,34,1,1,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,2,1,1,1,1,0,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,45,0,1,1,0,1,1,...,0,1,0,0,1,0,1,0,0,0
4,0,0,0,2,1,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [11]:
target = "Churn"
X = data.drop(target, axis= 1)
y = data.Churn

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)
X_train.shape

(5634, 27)

In [16]:
# Create Model 
 
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    return model

In [18]:
model = create_model()

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                448       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 593
Trainable params: 593
Non-trainable params: 0
_________________________________________________________________


In [22]:
results = model.fit(X_train, y_train, 
          validation_data = (X_test, y_test),
          epochs = 50,
          batch_size = 10) 

Epoch 1/50
564/564 [==============================] - 1s 1ms/step - loss: 8.1851 - accuracy: 0.7002 - val_loss: 0.9108 - val_accuracy: 0.6473
Epoch 2/50
564/564 [==============================] - 0s 778us/step - loss: 0.9592 - accuracy: 0.7430 - val_loss: 0.7325 - val_accuracy: 0.7999
Epoch 3/50
564/564 [==============================] - 0s 776us/step - loss: 1.0543 - accuracy: 0.7364 - val_loss: 1.4724 - val_accuracy: 0.7743
Epoch 4/50
564/564 [==============================] - 0s 775us/step - loss: 0.9891 - accuracy: 0.7412 - val_loss: 0.4567 - val_accuracy: 0.8070
Epoch 5/50
564/564 [==============================] - 0s 778us/step - loss: 0.8152 - accuracy: 0.7572 - val_loss: 3.1395 - val_accuracy: 0.4833
Epoch 6/50
564/564 [==============================] - 0s 776us/step - loss: 0.8961 - accuracy: 0.7469 - val_loss: 1.8978 - val_accuracy: 0.7828
Epoch 7/50
564/564 [==============================] - 0s 813us/step - loss: 0.9294 - accuracy: 0.7480 - val_loss: 0.4455 - val_accuracy: 0

### Hyper Parameter Tuning

In [32]:
# Batch size and epochs
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout


def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7832826137542724 using {'batch_size': 40, 'epochs': 20}
Means: 0.7788455963134766, Stdev: 0.030602254021777034 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7681846022605896, Stdev: 0.04014134767742437 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7832826137542724, Stdev: 0.013929782212755936 with: {'batch_size': 40, 'epochs': 20}
Means: 0.739802622795105, Stdev: 0.09741518448433523 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7483211278915405, Stdev: 0.06294237232974799 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7697951674461365, Stdev: 0.04214725179481711 with: {'batch_size': 100, 'epochs': 20}


In [39]:
# Optimizer

def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'optimizer': ['adam', 'nadam', 'sgd', 'adadelta', 'adagrad']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7580826640129089 using {'optimizer': 'adam'}
Means: 0.7580826640129089, Stdev: 0.051701910573436115 with: {'optimizer': 'adam'}
Means: 0.6788996398448944, Stdev: 0.13260030291519465 with: {'optimizer': 'nadam'}
Means: 0.7350045084953308, Stdev: 0.012946636225442116 with: {'optimizer': 'sgd'}
Means: 0.46455198526382446, Stdev: 0.23629299579711704 with: {'optimizer': 'adadelta'}
Means: 0.575909411907196, Stdev: 0.1770084627412355 with: {'optimizer': 'adagrad'}


In [40]:
# Learning Rate 

from tensorflow.keras.optimizers import Adam 

def create_model(lr=0.01):
    
    opt = Adam(learning_rate=lr)
    
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'lr': [0.001, 0.01, 0.05, 0.1, 0.25]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7777806520462036 using {'lr': 0.001}
Means: 0.7777806520462036, Stdev: 0.030370569413030735 with: {'lr': 0.001}
Means: 0.7348366618156433, Stdev: 0.12525371179693273 with: {'lr': 0.01}
Means: 0.7543479084968567, Stdev: 0.0264321967533735 with: {'lr': 0.05}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'lr': 0.1}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'lr': 0.25}


In [43]:
# Activation Functions 

def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'activation': ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7866531491279602 using {'activation': 'tanh'}
Means: 0.7557426929473877, Stdev: 0.07332933480840104 with: {'activation': 'softmax'}
Means: 0.6866907954216004, Stdev: 0.12166594453202906 with: {'activation': 'softplus'}
Means: 0.7256070613861084, Stdev: 0.07794058048423896 with: {'activation': 'softsign'}
Means: 0.7745869636535645, Stdev: 0.03168460924364308 with: {'activation': 'relu'}
Means: 0.7866531491279602, Stdev: 0.008165115072198897 with: {'activation': 'tanh'}
Means: 0.7493871688842774, Stdev: 0.08205160666465798 with: {'activation': 'sigmoid'}
Means: 0.7792009711265564, Stdev: 0.036881453561528846 with: {'activation': 'hard_sigmoid'}
Means: 0.7600326895713806, Stdev: 0.06404816016103745 with: {'activation': 'linear'}


In [44]:
# Network Weight Initialization 

def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(8, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'init_mode': ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7896707773208618 using {'init_mode': 'normal'}
Means: 0.7841615796089172, Stdev: 0.017575567080437456 with: {'init_mode': 'uniform'}
Means: 0.7882513999938965, Stdev: 0.016425218464713783 with: {'init_mode': 'lecun_uniform'}
Means: 0.7896707773208618, Stdev: 0.009553106161707016 with: {'init_mode': 'normal'}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'init_mode': 'zero'}
Means: 0.7848814964294434, Stdev: 0.026853257239458094 with: {'init_mode': 'glorot_normal'}
Means: 0.7041326940059662, Stdev: 0.12450036522253415 with: {'init_mode': 'glorot_uniform'}
Means: 0.7861213684082031, Stdev: 0.014997015741748321 with: {'init_mode': 'he_normal'}
Means: 0.7793792128562927, Stdev: 0.020737638293611808 with: {'init_mode': 'he_uniform'}


In [52]:
# Dropout Regularization 

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

def create_model(dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'weight_constraint': [1, 2, 3, 4, 5],
              'dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")              

Best: 0.8004969239234925 using {'dropout_rate': 0.3, 'weight_constraint': 1}
Means: 0.792155408859253, Stdev: 0.009809638666722197 with: {'dropout_rate': 0.0, 'weight_constraint': 1}
Means: 0.7642930388450623, Stdev: 0.052290362954945574 with: {'dropout_rate': 0.0, 'weight_constraint': 2}
Means: 0.79357430934906, Stdev: 0.012487072457986984 with: {'dropout_rate': 0.0, 'weight_constraint': 3}
Means: 0.7953503489494324, Stdev: 0.013311952034880116 with: {'dropout_rate': 0.0, 'weight_constraint': 4}
Means: 0.7955274939537048, Stdev: 0.008875422627034234 with: {'dropout_rate': 0.0, 'weight_constraint': 5}
Means: 0.7784905076026917, Stdev: 0.01924457444578992 with: {'dropout_rate': 0.1, 'weight_constraint': 1}
Means: 0.7733363628387451, Stdev: 0.024065703818745447 with: {'dropout_rate': 0.1, 'weight_constraint': 2}
Means: 0.7955275177955627, Stdev: 0.0103801220411747 with: {'dropout_rate': 0.1, 'weight_constraint': 3}
Means: 0.7921550989151, Stdev: 0.011362632257879882 with: {'dropout_rate'

In [53]:
# Neurons in Hidden Layer

def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=27, kernel_initializer='normal', kernel_constraint=MaxNorm(3), activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'neurons': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7964146494865417 using {'neurons': 5}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'neurons': 1}
Means: 0.7964146494865417, Stdev: 0.008557573777166875 with: {'neurons': 5}
Means: 0.794818127155304, Stdev: 0.004014619348988016 with: {'neurons': 10}
Means: 0.7799105167388916, Stdev: 0.02621004523076406 with: {'neurons': 15}
Means: 0.7953505277633667, Stdev: 0.011383858574595517 with: {'neurons': 20}
Means: 0.7953498840332032, Stdev: 0.01305632677204972 with: {'neurons': 25}
Means: 0.7845243692398072, Stdev: 0.020542722789990864 with: {'neurons': 30}
Means: 0.7850574016571045, Stdev: 0.025780031451786627 with: {'neurons': 35}
Means: 0.7942800760269165, Stdev: 0.01618888965889905 with: {'neurons': 40}
Means: 0.7937525749206543, Stdev: 0.022279291466279737 with: {'neurons': 45}
Means: 0.7836291909217834, Stdev: 0.019413342704277397 with: {'neurons': 50}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?